In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import Chroma
from langchain_community import embeddings
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import PromptTemplate

from langchain.output_parsers import PydanticOutputParser
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import BSHTMLLoader
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler


In [3]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
# from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel
from langchain import hub

In [4]:
llm = ChatOllama(model="mistral",temperature=0)

In [25]:
obj = hub.pull("wfh/proposal-indexing")
runnable = obj | llm

In [26]:
class Sentences(BaseModel):
    sentences: List[str]

In [27]:
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
def get_propositions(text):
    runnable_output = runnable.invoke({
    	"input": text
    }).content
    propositions = extraction_chain.invoke(runnable_output)["text"][0].sentences
    return propositions

In [29]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\\Users\\Barani\\Desktop\\local_ollama\\pdfs\\ACC_Data_merged.pdf")
pages = loader.load_and_split()

In [30]:
len(pages)

168

In [31]:
from agentic_chunker import AgenticChunker

In [32]:
chunker = AgenticChunker(llm)

In [33]:
chunker.add_proposition(pages)

Adding: '[Document(page_content="cancer.org | 1.800.227.2345\n \nWhat Is Cancer?\xa0\nCancer affects 1 in 3 people 
in the United States. Chances are that you or someone you\nknow has been affected by cancer. Here is some 
information to help you better\nunderstand what cancer is.\xa0\nCancer is more than just one disease●\nWhat is a 
tumor?●\nWhat is the difference between a benign and a malignant tumor?●\nWhat causes cancer?●\nWhat is the cancer 
stage?●\nHow does cancer spread?●\nYou are made up of trillions of cells that over your lifetime normally grow and 
divide as\nneeded. When cells are abnormal or get old, they usually die. Cancer starts when\nsomething goes wrong 
in this process and your cells keep making new cells and the old\nor abnormal ones don't die when they should. As 
the cancer cells grow out of control,\nthey can\xa0crowd out normal cells. This makes it hard for your body to work
the way it\nshould.\nFor many people, cancer can be treated successfully. In fact, more people than ever\nbefore 
are leading full lives after cancer treatment.\nCancer is more than just one disease\xa0\xa0\nThere are many types 
of cancer. Cancer can develop anywhere in the body and is\nnamed for the part of the body where it started. For 
instance, breast cancer that starts in\nthe breast is still called breast cancer even if it spreads (metastasizes) 
to other parts of\nthe body.\n1", metadata={'source': 
'C:\\Users\\Barani\\Desktop\\local_ollama\\pdfs\\ACC_Data_merged.pdf', 'page': 0}), Document(page_content="American
Cancer Society cancer.org | 1.800.227.2345 
____________________________________________________________________________________\nThere are two main categories
of cancer:\nHematologic (blood) cancers are cancers of the blood cells, including leukemia,\nlymphoma, and multiple
myeloma.●\nSolid tumor cancers are cancers of any of the other body organs or tissues. The\nmost common solid 
tumors are breast, prostate, lung, and colorectal cancers.●\nThese cancers are alike in some ways, but can be 
different in the ways they grow,\nspread, and respond to treatment. Some cancers grow and spread fast. Others 
grow\nmore slowly. Some are more likely to spread to other parts of the body. Others tend to\nstay where they 
started.\nSome types of cancer are best treated with surgery; others respond better to drugs such\nas chemotherapy.
Often 2 or more treatments are used to get the best results.\nWhat is a tumor?\xa0\xa0\nA tumor is a lump or growth
that can be benign or malignant (cancer).\nWhat is the difference between a benign and a malignant 
tumor?\xa0\xa0\nBenign tumors usually grow slowly, don't grow into tissues around them, and don't\ngrow into other 
parts of the body. They also don't often cause symptoms unless they\ngrow large enough to push on other structures.
If they need treatment, it is usually\nsurgery although radiation and medications can also be used. Benign tumors 
are not\ncancer.\nMalignant tumors (cancer) can grow quickly and uncontrollably. These tumors can\ngrow and spread 
into areas close by. Cells from these tumors can break away, travel\nthrough blood or lymph system, and begin to 
grow in other parts of the body. When this\nhappens it is called metastasis. Symptoms from malignant tumors usually
vary\ndepending on where the tumor is located. Malignant tumors are treated with surgery,\nradiation, and drugs 
such as chemotherapy.\nWhat causes cancer?\xa0\xa0\nCancer cells develop because of multiple changes in their 
genes. These changes can\nhave many possible causes1. Lifestyle habits, genes you get from your parents, and\nbeing
exposed to cancer-causing agents in the environment can all play a role. Many\n2", metadata={'source': 
'C:\\Users\\Barani\\Desktop\\local_ollama\\pdfs\\ACC_Data_merged.pdf', 'page': 1}), Document(page_content="American
Cancer Society cancer.org | 1.800.227.2345 
____________________________________________________________________________________\ntimes, there is no o

No chunks, creating a new one

Created new chunk (dcfc5):  Cancer Resources & Reliable Information (from American Cancer Society)

In [34]:
print(chunker.pretty_print_chunks())

You have 1 chunks

Chunk #0

Chunk ID: dcfc5

Summary:  It appears you have shared a list of references and resources related to cancer information and websites 
provided by the American Cancer Society. The documents include links to various organizations such as National 
Cancer Institute, Memorial Sloan-Kettering Cancer Center, National Institutes of Health, and Food and Drug 
Administration, among others. There are also references to articles on cancer treatment scams and evaluating online
sources of health information. The American Cancer Society is a reputable organization that provides reliable 
information about cancer and its treatments. The documents emphasize the importance of evaluating sources of health
information and caution against cancer treatment scams.

Propositions:

-[Document(page_content="cancer.org | 1.800.227.2345\n \nWhat Is Cancer?\xa0\nCancer affects 1 in 3 people in 
the United States. Chances are that you or someone you\nknow has been affected by cancer. Here is some information 
to help you better\nunderstand what cancer is.\xa0\nCancer is more than just one disease●\nWhat is a tumor?●\nWhat 
is the difference between a benign and a malignant tumor?●\nWhat causes cancer?●\nWhat is the cancer stage?●\nHow 
does cancer spread?●\nYou are made up of trillions of cells that over your lifetime normally grow and divide 
as\nneeded. When cells are abnormal or get old, they usually die. Cancer starts when\nsomething goes wrong in this 
process and your cells keep making new cells and the old\nor abnormal ones don't die when they should. As the 
cancer cells grow out of control,\nthey can\xa0crowd out normal cells. This makes it hard for your body to work the
way it\nshould.\nFor many people, cancer can be treated successfully. In fact, more people than ever\nbefore are 
leading full lives after cancer treatment.\nCancer is more than just one disease\xa0\xa0\nThere are many types of 
cancer. Cancer can develop anywhere in the body and is\nnamed for the part of the body where it started. For 
instance, breast cancer that starts in\nthe breast is still called breast cancer even if it spreads (metastasizes) 
to other parts of\nthe body.\n1", metadata={'source': 
'C:\\Users\\Barani\\Desktop\\local_ollama\\pdfs\\ACC_Data_merged.pdf', 'page': 0}), Document(page_content="American
Cancer Society cancer.org | 1.800.227.2345 
____________________________________________________________________________________\nThere are two main categories
of cancer:\nHematologic (blood) cancers are cancers of the blood cells, including leukemia,\nlymphoma, and multiple
myeloma.●\nSolid tumor cancers are cancers of any of the other body organs or tissues. The\nmost common solid 
tumors are breast, prostate, lung, and colorectal cancers.●\nThese cancers are alike in some ways, but can be 
different in the ways they grow,\nspread, and respond to treatment. Some cancers grow and spread fast. Others 
grow\nmore slowly. Some are more likely to spread to other parts of the body. Others tend to\nstay where they 
started.\nSome types of cancer are best treated with surgery; others respond better to drugs such\nas chemotherapy.
Often 2 or more treatments are used to get the best results.\nWhat is a tumor?\xa0\xa0\nA tumor is a lump or growth
that can be benign or malignant (cancer).\nWhat is the difference between a benign and a malignant 
tumor?\xa0\xa0\nBenign tumors usually grow slowly, don't grow into tissues around them, and don't\ngrow into other 
parts of the body. They also don't often cause symptoms unless they\ngrow large enough to push on other structures.
If they need treatment, it is usually\nsurgery although radiation and medications can also be used. Benign tumors 
are not\ncancer.\nMalignant tumors (cancer) can grow quickly and uncontrollably. These tumors can\ngrow and spread 
into areas close by. Cells from these tumors can break away, travel\nthrough blood or lymph system, and begin to 
grow in other parts of the body. When this\nhappens it is called metastasis. Symptoms from malignant tumors usually
vary\ndepending on where the tumor is located. Malignant tumors are treated with surgery,\nradiation, and drugs 
such as chemotherapy.\nWhat causes cancer?\xa0\xa0\nCancer cells develop because of multiple changes in their 
genes. These changes can\nhave many possible causes1. Lifestyle habits, genes you get from your parents, and\nbeing
exposed to cancer-causing agents in the environment can all play a role. Many\n2", metadata={'source': 
'C:\\Users\\Barani\\Desktop\\local_ollama\\pdfs\\ACC_Data_merged.pdf', 'page': 1}), Document(page_content="American
Cancer Society cancer.org | 1.800.227.2345 
____________________________________________________________________________________\ntimes, there is no obvious 


None


In [35]:
chunks = chunker.get_chunks(get_type='list_of_strings')
print(chunks)

TypeError: sequence item 0: expected str instance, list found